In [1]:
from __future__ import print_function
import sys
import os
import argparse
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
from torch.autograd import Variable
from data import *
from PIL import Image
from utils.augmentations import SSDAugmentation

import torch.utils.data as data
from ssd import build_ssd

/home/broiron/.local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### check cuda

In [2]:
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
save_folder = os.path.join(os.getcwd(), 'test')
if not os.path.exists(save_folder):
    os.mkdir(save_folder)

In [4]:
def test_net(save_folder, net, cuda, testset, transform, thresh):
    # dump predictions and assoc. ground truth to text file for now
    filename = os.path.join(save_folder, 'test1.txt')
    num_images = len(testset)
    for i in range(num_images):
        print('Testing image {:d}/{:d}....'.format(i+1, num_images))
        img, annotation, _, _, img_id = testset.pull_item(i)
        # img, annotation, h, w = testset.pull_item(i)
        print(img_id)

        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        print(device)
        
        x = img.unsqueeze(0)
        x = x.type(torch.FloatTensor)
        x = x.to(device)
        x = Variable(x)

        with open(filename, mode='a') as f:
            f.write('\nGROUND TRUTH FOR: '+str(img_id)+'\n')
            for box in annotation:
                f.write('label: '+' || '.join(str(b) for b in box)+'\n')

        net = net.to(device)
        y = net(x)      # forward pass
        detections = y.data
        # scale each detection back up to the image
        scale = torch.Tensor([img.shape[1], img.shape[0],
                             img.shape[1], img.shape[0]])
        pred_num = 0
        for i in range(detections.size(1)):
            j = 0
            while detections[0, i, j, 0] >= args.visual_threshold:
                if pred_num == 0:
                    with open(filename, mode='a') as f:
                        f.write('PREDICTIONS: '+'\n')
                score = detections[0, i, j, 0]
                label_name = CUSTOM_CLASSES[i-1]
                pt = (detections[0, i, j, 1:]*scale).cpu().numpy()
                coords = (pt[0], pt[1], pt[2], pt[3])
                pred_num += 1
                with open(filename, mode='a') as f:
                    f.write(str(pred_num)+' label: '+label_name+' score: ' +
                            str(score) + ' '+' || '.join(str(c) for c in coords) + '\n')
                print(str(pred_num)+' label: '+label_name+' score: ' +
                            str(score) + ' '+' || '.join(str(c) for c in coords))
                j += 1

In [5]:
net = build_ssd(phase='test', size=300, num_classes=2)
net.load_state_dict(torch.load('./weights/linedataset_vol1_1a.pth'))
net.eval()
print("Finished loading model!")

testset = COCODetection(root='/home/broiron/broiron/line_dataset_vol1_coco/', 
                        transform=SSDAugmentation(300, MEANS), image_set='test')

test_net(save_folder, net, True, testset, BaseTransform(net.size, (104, 117, 123)), 0.6)


Finished loading model!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Testing image 1/100....
5
cuda:0


/home/broiron/broiron/model_train/ssd-pytorch-custom/utils/augmentations.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)


TypeError: forward() missing 1 required positional argument: 'x'